In [3]:
from tinydb import TinyDB
import json

# I worked very hard and built this from the ground up.
# My goal was to put back the class system in the end.
# But I am out of time for this week.
# I got alot of dictionary practice but I still need to work on these skills.

db = TinyDB('db.json')

# load people
with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/people.json', 'rb') as file:
    people = json.load(file)
    file.close()

# load visited
with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/visited.json', 'rb') as file:
    visited = json.load(file)
    file.close()

# load measurements
with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/measurements.json', 'rb') as file:
    measurements = json.load(file)
    file.close()

# load sites
with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/sites.json', 'rb') as file:
    sites = json.load(file)
    file.close()

# This time I will try with dictionaries
out = []
dataset = {}

print('You will see my progression in thought\n(Please scroll to the bottom to see the final outputs)\n')

# I decided to start with visits since site can be added easily
# I ended up working backwards through what I needed
# this was pretty straight forward
for key, visit in visited.items():
    site_id = visit['site_id']
    print(site_id)
    visit_id = visit['visit_id']
    print(visit_id)
    dataset[visit_id] = visit
    print(dataset[visit_id])
    if site_id in sites:
        dataset[visit_id]['site'] = sites[site_id]
    else:
        # we have to account for the person that never visited a site
        {}

# I now have my visit dictionary with sites
visited = dataset

print('\nOutput after making my change to visits.\n'
      'As you can see I replaced the keys to something I could use later.\n'
      'I honestly was not happy with the composite keys since the information was already at our fingertips\n')
print(json.dumps(visited,indent=2))

# here I am linking visits to measurements
for key, measurement in measurements.items():
    visit_id = measurement['visit_id']
    if visit_id in visited:
        # this helps me with people who did not have measurments
        if 'measurements' not in visited[visit_id]:
            visited[visit_id]['measurements'] = []
        else:
            visited[visit_id]['measurements']

        if 'persons' not in visited[visit_id]:
            # danforth made me account for members that had no visits
            visited[visit_id]['persons'] = set()
        else:
            visited[visit_id]['persons']

        # now that I have them I need to build them into my list
        visited[visit_id]['measurements'].append(measurement)
        # I decided to track the person here
        visited[visit_id]['persons'].add( measurement['person_id'])

# json.dumps needed a list item according to
# https://stackoverflow.com/questions/22281059/set-object-is-not-json-serializable
# I believe this was happening because of my list items.
# It helped me print it so I could see what I had.
# I really had to step through this problem to figure it out

def set_default(obj):
    if isinstance(obj, set):
        return list(obj)
    raise TypeError

print('\nI added measurements to the visit site info.\n')
print(json.dumps(visited,indent=2,default=set_default))


# final step I need to assign to the different people using what I have in both visits and measurement

for person_id, person in people.items():
    # Here I am going to put it all together
    # I grab all people and put them in a dictionary
    data = {}
    data['person_id'] = person['person_id']
    data['personal_name'] = person['personal_name']
    data['family_name'] = person['family_name']
    # I make a spot for visits
    data['visits'] = []

    # I have already built visited now its time to fill it in
    for visit in visited.values():
        if 'persons' in visit and person_id in visit['persons']:
            visit_data = {}
            visit_data['visit_id'] = visit['visit_id']
            visit_data['site_id'] = visit['site_id']
            visit_data['visit_date'] = visit['visit_date']
            visit_data['site'] = visit['site']
            # I make a spot for measurements
            visit_data['measurements'] = []

            # I have built measurement already I just need to see who the measurement belongs to
            for measurement in visit['measurements']:
                if measurement['person_id'] == person_id:
                    visit_data['measurements'].append(measurement)
            data['visits'].append(visit_data)
    out.append(data)

print('\n ------------------------------------------------------------------------------\n')
print('\n\tThe Final Result!!!\n')
print('\n ------------------------------------------------------------------------------\n')
print(json.dumps( out, indent=2 ))

with open('/home/totennacht/Bellevue/dsc650/dsc650/assignments/assignment02/results/kvdb/patient-info.json', 'w') as outfile:
    json.dump(out, outfile)
    outfile.close()

You will see my progression in thought
(Please scroll to the bottom to see the final outputs)

DR-1
619
{'visit_id': 619, 'site_id': 'DR-1', 'visit_date': '1927-02-08'}
DR-1
622
{'visit_id': 622, 'site_id': 'DR-1', 'visit_date': '1927-02-10'}
DR-3
734
{'visit_id': 734, 'site_id': 'DR-3', 'visit_date': '1930-01-07'}
DR-3
735
{'visit_id': 735, 'site_id': 'DR-3', 'visit_date': '1930-01-12'}
DR-3
751
{'visit_id': 751, 'site_id': 'DR-3', 'visit_date': '1930-02-26'}
DR-3
752
{'visit_id': 752, 'site_id': 'DR-3', 'visit_date': nan}
MSK-4
837
{'visit_id': 837, 'site_id': 'MSK-4', 'visit_date': '1932-01-14'}
DR-1
844
{'visit_id': 844, 'site_id': 'DR-1', 'visit_date': '1932-03-22'}

Output after making my change to visits.
As you can see I replaced the keys to something I could use later.
I honestly was not happy with the composite keys since the information was already at our fingertips

{
  "619": {
    "visit_id": 619,
    "site_id": "DR-1",
    "visit_date": "1927-02-08",
    "site": {
      